In [ ]:
import os
import glob
import time
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data_path = "../data/dog-breed/"

In [ ]:
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

In [ ]:
label_table = pd.read_csv(os.path.join(data_path, "labels.csv"))
samp = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [ ]:
label_table.shape

In [ ]:
label_table.head()

In [ ]:
label2idx = dict((label, idx) for idx, label in enumerate(label_table.breed.unique()))
idx2label = dict((idx, label) for idx, label in enumerate(label_table.breed.unique()))

In [ ]:
len(label2idx)

In [ ]:
label_table["breed_id"] = label_table.breed.map(label2idx)

In [ ]:
label_table.shape

In [ ]:
label_table.head()

In [ ]:
# label, freq = np.unique(label_table.breed_id.values, return_counts=True)

In [ ]:
# plt.figure(figsize=(15, 5))
# plt.bar(label, freq)

### preview

In [ ]:
# fig, ax = plt.subplots(3, 7, figsize=(15, 6))
# r = 0
# c = 0
# for idx in label_table.index[:21]:
#     img = plt.imread(os.path.join(train_path, label_table.loc[idx, "id"] + ".jpg"))
    
#     ax[r, c].imshow(img)
#     ax[r, c].axis("off")
#     ax[r, c].set_title(label_table.loc[idx, "breed"])
    
#     c += 1
#     if c % 7 == 0:
#         c = 0 
#         r += 1

In [ ]:
# img_dim = {"path": [], "h": [], "w": [], "c": []}
# for idx in tqdm(label_table.index):
#     img = plt.imread(os.path.join(train_path, label_table.loc[idx, "id"] + ".jpg"))
#     h, w, c = img.shape
#     img_dim["path"].append(label_table.loc[idx, "id"])
#     img_dim["h"].append(h)
#     img_dim["w"].append(w)
#     img_dim["c"].append(c)

In [ ]:
# d_img_dim = pd.DataFrame(img_dim)

# d_img_dim.describe()

# sns.boxplot(img_dim["h"])

# sns.boxplot(img_dim["w"])

### resize 

In [ ]:
# def img_scaling(h, w):
#     max_pxl = 300
    
#     if h > w:
#         ratio = max_pxl / h
#     else:
#         ratio = max_pxl / w
    
#     new_height = h * ratio
#     new_width = w * ratio
        
#     return  new_height, new_width

In [ ]:
# for idx in d_img_dim.index[:5]:
#     fig, ax = plt.subplots(1, 2)
#     img = plt.imread(os.path.join(train_path, label_table.loc[idx, "id"] + ".jpg"))
#     ax[0].imshow(img)
#     ax[0].set_title(str(img.shape))
    
#     h, w, _ = img.shape
    
#     h_new, w_new = img_scaling(h, w)
#     img_re = cv2.resize(img, (int(w_new), int(h_new)))
#     ax[1].imshow(img_re)
#     ax[1].set_title(str(img_re.shape))
#     plt.show()

## datasets

In [ ]:
label_table["id"] = label_table.id.apply(lambda x: os.path.join(train_path, x + ".jpg"))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(label_table.id.values, label_table.breed_id.values)

In [ ]:
class BreedDataset(Dataset):
    def __init__(self, X_train, X_test, y_train, y_test):
        self.dataset = {
            "train": (X_train, y_train, len(X_train)),
            "test": (X_test, y_test, len(X_test))
        }
        self.set_split("train")
        
    def set_split(self, split="train"):
        self.x, self.target, self.length = self.dataset[split]
    
    def img_scaling(self, img):
        max_pxl = 300
        h, w, c = img.shape
        if h > w:
            ratio = max_pxl / h
        else:
            ratio = max_pxl / w

        new_height = h * ratio
        new_width = w * ratio
        
        img_re = cv2.resize(img, (int(new_width), int(new_height)))
        
        return  img_re
    
    def padding(self, img_arr):
        x = np.zeros((300, 300, 3))
        h, w, c = img_arr.shape
        x[:h,:w,:] = img_arr
        
        return x
    
    def read_image(self, path):
        img = plt.imread(path, 0)
        img = img / 255
        img = self.img_scaling(img)
        img = self.padding(img)
        img = np.transpose(img, (2, 0, 1))
        return img
    
    
    def __getitem__(self, index):
        x = self.x[index]
        x = self.read_image(x)
        x = torch.Tensor(x)
        
        y = torch.LongTensor([self.target[index]])
        
        return x, y
    
    def __len__(self):
        return self.length

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 1, kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3),
            nn.ReLU(),
            nn.Flatten()
        )
        
        self.fc1 = nn.Linear(5041, 512)
        self.fc2 = nn.Linear(512, 120)
        
    def forward(self, input_):
        out = self.network(input_)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out

In [ ]:
dataset = BreedDataset(X_train, X_test, y_train, y_test)

In [ ]:
# model = Classifier().to(device)
model = torchvision.models.inception_v3(pretrained=True, progress=True, aux_logits = False)

In [ ]:
model.fc = nn.Linear(2048, 120)

In [ ]:
model = model.to(device)

In [ ]:
num_param = sum(p.numel() for p in model.parameters())

In [ ]:
print(f"total number of parameters {num_param:,}")

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
sm = nn.Softmax(dim=1)

In [ ]:
def compute_accuracy(out, y):
    out = sm(out)
    n_indicies = out.argmax(1).long()
    n_correct = torch.eq(n_indicies, y).sum().item()
    accuracy = (n_correct / y.shape[0]) * 100
    
    return accuracy    

In [ ]:
def compute_duration(start, end):
    duration = end - start
    m = int(duration / 60)
    s = int(duration % 60)
    
    return m, s

In [ ]:
history_dict = {
    "running_loss": [],
    "running_loss_v": [],
    "running_acc": [],
    "running_acc_v": []
}

In [ ]:
batchsize = 64
epochs = 101

In [ ]:
for epoch in range(1, epochs):
    running_loss = 0
    running_loss_v = 0
    running_acc = 0
    running_acc_v = 0
    
    start = time.perf_counter()
    
    dataset.set_split("train")
    data_gen = DataLoader(dataset, batch_size = batchsize)
    model.train()
    for batch_index, (x, y) in enumerate(data_gen, 1):
        
        optimizer.zero_grad()
        
        x = x.to(device)
        y = y.squeeze(1)
        y = y.to(device)
        
        out = model(x)
        
        loss = criterion(out, y)
        loss.backward()
        loss_ = loss.item()
        running_loss += (loss_ - running_loss) / batch_index
        
        accuracy = compute_accuracy(out, y)
        running_acc += (accuracy - running_acc) / batch_index
        
        optimizer.step()
        
    dataset.set_split("test")
    data_gen = DataLoader(dataset, batch_size = batchsize)
    model.eval()
    for batch_index, (x, y) in enumerate(data_gen, 1):
        
        x = x.to(device)
        y = y.squeeze(1)
        y = y.to(device)
        
        out = model(x)
        
        loss = criterion(out, y)
        loss_ = loss.item()
        running_loss_v += (loss_ - running_loss_v) / batch_index
        
        accuracy = compute_accuracy(out, y)
        running_acc_v += (accuracy - running_acc_v) / batch_index
        
    end = time.perf_counter()
    m, s = compute_duration(start, end)
    print(f"epoch: {epoch} | {m} m {s} s")
    print(f"\ttrain loss {running_loss:.2f} | accuracy: {running_acc:.2f}")
    print(f"\tval loss {running_loss_v:.2f} | accuracy: {running_acc_v:.2f}")
    
    history_dict["running_loss"].append(running_loss)
    history_dict["running_loss_v"].append(running_loss_v)
    history_dict["running_acc"].append(running_acc)
    history_dict["running_acc_v"].append(running_acc_v)

In [ ]:
pickle.dump(history_dict, open("history_dog_inception_v3.pkl", "wb"))
torch.save((model.state_dict(), "model_dog_inception_v3.pt"))